<a href="https://colab.research.google.com/github/ayuadmy/simpasi/blob/main/SIMPASI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Needed


In [ ]:
!python --version

Python 3.7.13


In [ ]:
pip install -q tensorflow-recommenders

In [ ]:
import pathlib
import os
import pprint
import tempfile

from typing import Dict, Text

import csv
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# MPASI Menu, Recipe, Ingredients Dataset

In [ ]:
#Download datasets from google drive
!gdown --id 1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS\

#Move files to tmp (To be deleted after closing this google colab in some time)
!mv "/content/bahan_makanan.csv" "/tmp"

#Create directory of datasets
bahan_makanan_dir = "/tmp/bahan_makanan.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS
To: /content/bahan_makanan.csv
100% 5.64k/5.64k [00:00<00:00, 3.74MB/s]


In [ ]:
!gdown --id 1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT\

!mv "/content/resep.csv" "/tmp"
resep_dir = "/tmp/resep.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT
To: /content/resep.csv
100% 3.57k/3.57k [00:00<00:00, 1.97MB/s]


In [ ]:
!gdown --id 1B5PuaDeqQljS4r1YKRluvPs14BDsautc\

!mv "/content/komposisi.csv" "/tmp"
komposisi_dir = "/tmp/komposisi.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1B5PuaDeqQljS4r1YKRluvPs14BDsautc
To: /content/komposisi.csv
100% 4.40k/4.40k [00:00<00:00, 6.82MB/s]


In [ ]:
!gdown --id 1Elz56RH18uKtrV3DDePpT9SKOzhFGVPb\

!mv "/content/kombinasi_menu.csv" "/tmp"
kombinasi_menu_dir = "/tmp/kombinasi_menu.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Elz56RH18uKtrV3DDePpT9SKOzhFGVPb
To: /content/kombinasi_menu.csv
100% 3.00M/3.00M [00:00<00:00, 140MB/s]


In [ ]:
import pandas as pd

bahan_makanan = pd.read_csv(bahan_makanan_dir)
bahan_makanan.head()

,id_bahan,nama_bahan
0,0,Agar-Agar
1,1,Agar Laut
2,2,Akar Tonjong
3,3,Alpukat
4,4,Ampas Tahu


In [ ]:
kombinasi_menu = pd.read_csv(kombinasi_menu_dir)

kombinasi_menu["malam"].fillna("", inplace = True) 
kombinasi_menu["snack1"].fillna("", inplace = True) 
kombinasi_menu["snack2"].fillna("", inplace = True) 

kombinasi_menu.head()

,id_kombinasi,pagi,siang,malam,snack1,snack2,bulan_6,bulan_7_8,bulan_9_11,bulan_12_24,total_energi,total_carbo,total_protein,total_lipid
0,1,Apel Rebus,Apel Rebus,-,-,-,1,0,0,0,144.00,38.12,0.72,0.46
1,2,Bubur Apel,Bubur Apel,-,-,-,1,0,0,0,144.50,34.52,0.66,0.42
2,3,Bubur Kacang Polong,Bubur Kacang Polong,-,-,-,1,0,0,0,127.44,15.60,3.84,5.52
3,4,Bubur Kembang Kol,Bubur Kembang Kol,-,-,-,1,0,0,0,143.20,21.12,9.46,2.32
4,5,Bubur Plum,Bubur Plum,-,-,-,1,0,0,0,128.10,30.88,1.10,0.02


In [ ]:
import csv

dict_komposisi = {}

with open(komposisi_dir, mode="r") as data_komposisi:
  reader = csv.reader(data_komposisi)
  keys = next(reader)
  for row in reader:
    if row[0] not in dict_komposisi.keys():
      dict_komposisi[row[0]] = []
    dict_komposisi[row[0]].append(row[1])

data = {keys[0]: dict_komposisi.keys(), keys[1]: dict_komposisi.values()}
komposisi = pd.DataFrame.from_dict(data)
komposisi.head()

,id_resep,id_bahan
0,0,"[63, 264, 0]"
1,1,[9]
2,2,"[9, 262, 279]"
3,3,"[326, 290, 9]"
4,4,"[143, 362, 202, 49, 20, 114]"


In [ ]:
resep = pd.read_csv(resep_dir)
resep.head()

,id_resep,nama_resep
0,0,Agar-agar Isi Blewah
1,1,Apel Rebus
2,2,Apple Crumble
3,3,Apricot Tahu
4,4,Ayam Zukini Hati


# Simulasi Aplikasi SIMPASI - User Baru

Pada user baru, customer akan melewati tahap-tahap berikut akan berupa:

1.   Log-in/Sign-up dengan email dan passsword
2.   Meminta data bayi berupa nama, tanggal lahir, bahan yang pernah dicoba sebelumnya (suka/tidak suka/alergi)
3.   Mendapatkan rekomendasi makanan dari riwayat makanan yang disukai/tidak disukai/alergi




Sehingga, sistem rekomendasi harus melakukan:


1.   Meminta username/email/password untuk masuk ke akun
2.   Meminta nama, tanggal lahir, bahan yang pernah dicoba sebelumnya *(suka/tidak suka/alergi)
3.   Menghitung umur anak
4.   Menghitung EER anak
5.   Memfilter bahan berdasarkan alergi
6.   Memfilter resep berdasarkan umur dan EER
7.   Menampilkan kombinasi menu dari resep yang cocok




## I. *Contoh Profil User*

In [ ]:
#Contoh
username = "brianjang"
password = "d8d807375d408c888d0ff9974a82edf4971ed91c0810ec4daa5e47b91bf0de76"
email = "ujang@mail.com"
nama_user = "Ujang Ramabrian"
nama_bayi = "Biyan"
tanggal_lahir = "2021/07/03"
berat_badan = 6.9

## II. Cek 8 Bahan (Suka, Tidak Suka, Alergi, Tidak tahu)


In [ ]:
#Menampilkan Nama Bahan Berdasarkan ID
def ingredient_name(id):
  return bahan_makanan.loc[id, 'nama_bahan']

#driver code
print(ingredient_name(1))

Agar Laut


In [ ]:
#Bahan Alergi Umum: Susu, Telur Ayam, Telur Bebek, Kacang Tanah, Kacang Kedelai, Udang, Kerang, Cumi-Cumi, Tepung Beras, Roti, Daging Sapi, Pisang, Alpukat
tes_bahan = [319, 334, 336, 199, 195, 356, 102, 223, 341, 298, 112, 290, 3]
jumlah_tes = 8
alergi = []
tidak_suka = []
suka = []
tidak_tahu = []

import random
tes_bahan_random = random.sample(tes_bahan, jumlah_tes)

#Meminta 8 Feedback Bahan dari User
print("Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan")
print("Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu")
for i in range(len(tes_bahan_random)):
  nama_bahan = ingredient_name(tes_bahan_random[i])
  
  x = int(input(nama_bahan + ": "))

  if x == 2:
    tidak_suka.append(tes_bahan_random[i])
  elif x == 3:
    alergi.append(tes_bahan_random[i])
  elif x == 4:
    tidak_tahu.append(tes_bahan_random[i])
  elif x == 1:
    suka.append(tes_bahan_random[i])


Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan
Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu


KeyboardInterrupt: ignored

## III. *Hitung Umur dan EER*

In [ ]:
from datetime import datetime
from datetime import date

#Fungsi Menghitung Umur User
def ageCalculator(birthdate):
  my_birthdate = datetime.strptime(birthdate, "%Y/%m/%d")
  my_year = my_birthdate.year
  my_month = my_birthdate.month
  my_day = my_birthdate.year
  
  born = my_birthdate.date()
  today = date.today()
  
  years = ((today-born).total_seconds()/ (365.242*24*3600))
  yearsInt = int(years)
  months = (years-yearsInt)*12
  monthsInt = int(months)
  days = (months-monthsInt)*(365.242/12)
  daysInt = int(days)

  return yearsInt, monthsInt, daysInt
  

#Fungsi Menghitung EER (Estimated Energy Requirements)
def eerCalculator(year, month, day, weight):
  if month == 6:
    EER = (89*weight - 100) + 56
  elif 7 <= month <= 12:
    EER = (89*weight - 100) + 22
  elif 13 <= month <= 35:
    EER = (89*weight - 100) + 20
  
  return EER
  
#Driver Code
#Cari Umur User (Tahun, Bulan, Tanggal)
tahun, bulan, hari = ageCalculator(tanggal_lahir)
print("Sekarang kamu berumur {0} tahun, {1} bulan, {2} hari.".format(tahun , bulan, hari))

#Cari EER User (+-10%)
eer = eerCalculator(tahun, bulan, hari, berat_badan)
print("Kebutuhan energi harianmu: {0} - {1} Joule".format(int(eer*0.9), int(eer*1.1)))

## IV. *Filtering Resep Berdasarkan Alergi*

## V. *Filtering Resep Berdasarkan umur dan EER*

## VI. *Model Sistem Rekomendasi Menu MPASI*